# Content Statistics

In [ ]:
import sys
import numpy as np
import numpy.matlib
import numpy.random
import ipywidgets as widgets
import scipy.io
import skimage
import imageio
import colour as c
import math
import csv
import ipywidgets as widgets
import cv2
import time
import re
import pandas as pd
import numpy as np
import os

from mpl_toolkits.mplot3d import axes3d 
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy import ndimage
from scipy import misc
from scipy import fftpack
from scipy import optimize
from datetime import datetime
from matplotlib.patches import Rectangle
from fnmatch import fnmatch

In [ ]:
## Linear to sRGB
def lin2srgb (x):
    result = (x <= 0.0031308) * np.multiply(x,12.92) + (x > 0.0031308) * np.subtract(np.multiply(1.055,np.power(x,(1/2.4))),0.055)
    return result


## sRGB to Linear 
def srgb2lin (x):
    result = (x <= 0.04045) * np.divide(x,12.92) + (x > 0.04045) * np.power(np.divide(np.add(x,0.055),1.055),2.4)
    return result

# SDR Formulas

## Input Luminance between 0-1
## Output Videosignal
def oetf709(x): 
    return np.subtract(np.multiply(np.power(x, 0.45), 1.099), 0.099)*(0.018<= x) + np.multiply(4.5, x) * (x <0.018)

def eotf1886(x, Lw = 100, Lb = 0): 
    gamma = 2.4
    a = np.power(np.subtract(np.power(Lw, np.divide(1,gamma)),np.power(Lb, np.divide(1, gamma))), gamma)
    b = np.divide(np.power(Lb, np.divide(1,gamma)) , np.subtract(np.power(Lw , np.divide(1,gamma)), np.power( Lb , np.divide(1, gamma))))
    return np.multiply(a , np.power( np.maximum(np.add(x, b), 0) , gamma))

def eotf1886_CRT(x, Lw = 100, Lb = 0, b = 0): 
    Vc = 0.35
    a1 = 2.6
    a2 = 3.0
    k = np.subtract(Lw,np.power(np.add(1,b),a1))
    return (x < Vc) * np.multiply(np.multiply(k,np.power(np.add(Vc,b),np.subtract(a1,a2))),np.power(np.add(x,b),a2)) + (Vc <= x) * np.multiply(k,np.power(np.add(x,b),a1))

## Luma Formulas for Rec2020 / Rec709 -> Fixed Addition
def luma709(img): 
    return np.multiply(img[:,:,0],0.2126) + np.multiply(img[:,:,1],0.7152) + np.multiply(img[:,:,2],0.0722)
def luma2020(img): 
    return np.multiply(img[:,:,0],0.2627) + np.multiply(img[:,:,1],0.6780) + np.multiply(img[:,:,2],0.0593)

## Formulas for PQ

# increasing rec.709 formulas 
def steroids709(x): 
    return np.subtract(np.multiply(np.power(np.multiply(59.5208 , x), 0.45), 1.099), 0.099)*(0.0003024< x) + np.multiply(267.84, x) * (x <= 0.0003024)

def steroids1886(x): 
    return np.multiply(100, np.power(x, 2.4))


## Transportfunctions

def pq_eotf(x): 
    ##m1
    m1 = np.divide(2610, 16384) ## 2^14 = 16384
    m2 = np.multiply(np.divide(2523, 4096), 128) ##2^12 = 4096 , 2^7 = 128 -> Jan fragen
    c1 = np.divide(3424, 4096)
    c2 = np.multiply(np.divide(2413, 4096), 32)
    c3 = np.multiply(np.divide(2392, 4096), 32)

    
    return np.multiply(np.power(np.divide(np.maximum( np.subtract( np.power(x , np.divide(1 , m2)), c1) , 0) , np.subtract(c2, np.multiply(c3 , np.power(x , np.divide(1 , m2))))) , np.divide(1, m1)), 10000)

## Inverse_eotf

def pq_eotf_inv(x): 
    m1 = np.divide(2610, 16384) ## 2^14 = 16384
    m2 = np.multiply(np.divide(2523, 4096), 128) ##2^12 = 4096 , 2^7 = 128 -> Jan fragen
    c1 = np.divide(3424, 4096)
    c2 = np.multiply(np.divide(2413, 4096), 32)
    c3 = np.multiply(np.divide(2392, 4096), 32)

    y = np.divide(x , 10000) ## Normalisierung auf 0.00 bis 1.0

    return np.power( np.divide(np.add(c1 , np.multiply(c2 , np.power(y , m1))), np.add(1 , np.multiply(c3 , np.power(y , m1)))), m2)

# RGB Input
def hlg_oetf(x): 
    a = 0.17883277
    b = np.subtract(1 , np.multiply(4 , a))
    c = np.subtract(0.5 , np.multiply(a , np.log(np.multiply(4 , a))))

    return np.where(x <= np.divide(1,12), np.sqrt(np.multiply(3,x )), np.add( np.multiply(a , np.log(np.subtract( np.multiply(12 , x ), b))) , c) )

def hlg_oetf_inv(x):
    a = 0.17883277
    b = np.subtract(1 , np.multiply(4 , a))
    c = np.subtract(0.5 , np.multiply(a , np.log(np.multiply(4 , a))))
    
    return np.where(x <= 0.5 , np.divide( np.power(x,2), 3), np.divide(np.add(np.exp(np.divide(np.subtract(x , c), a)), b) , 12) )

## Lw = Usergain, nominal peak Luminance of the display (achromatic)
def hlg_ootf(x, Lw): 
    r = x[: , : , 0]
    g = x[: , : , 1]
    b = x[: , : , 2]
    luma = np.add(np.multiply(0.2627 ,r ), np.multiply(0.678 , g), np.multiply(0.0593 , b))
    alpha = Lw
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    rd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), r)
    gd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), g)
    bd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), b)
    return np.stack((rd , gd , bd))

## Lw = Usergain, nominal peak Luminance of the display (achromatic)
def hlg_ootf_bw(x, Lw=1000): 
    alpha = Lw
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    return np.multiply(np.multiply( alpha , np.power(x , np.subtract(gamma , 1))),x)

def hlg_eotf(x, Lw = 1000, Lb = 0.005):
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    beta = np.sqrt(np.multiply(3 , np.power(np.divide(Lb , Lw), np.divide(1,gamma))))
    return hlg_ootf_bw(hlg_oetf_inv(np.maximum(0 , (np.add( np.multiply( np.subtract(1 , beta), x ), beta)))), Lw)


def slog3oetf(x): 
    # reflection add
    #vx = x*0.9

    return (x < 0.01125)* np.divide( np.add( np.multiply(x , np.divide(np.subtract(171.2102946929, 95), 0.01125)), 95), 1023) + (x >= 0.01125)* np.divide(np.add(420 , np.multiply( np.log10(np.divide(np.add(x,0.01), np.add(0.18, 0.01))), 261.5)), 1023)

def slog3eotf(x): 
    return (x >= np.divide(171.2102946929,1023)) * np.subtract( np.multiply(np.power(10 , np.divide( np.subtract(420 , np.multiply(x , 1023)) , 261.5)) , np.add(0.18 , 0.01)) , 0.01) + (x < np.divide(171.2102946929,1023)) * np.multiply(np.subtract(np.multiply(x , 1023), 95 ) , np.divide(0.01125 , np.subtract(171.2102946929 , 95)))

# These are the ARRI LogCv3 Formulas
def lin2logc(x):
    return np.multiply(x > 0.010591, 0.24719 * np.log10(5.555556 * x + 0.052272) + 0.385537) + np.multiply(x <= 0.010591, 5.367655 * x + 0.092809)
def logc2lin(x):
    return np.multiply(x > 0.149659, (np.power(10.0, (x - 0.385537) / 0.247189) - 0.052272) / 5.555556) + np.multiply(x <= 0.149659, (x - 0.092809) / 5.367650)

### LOG C4
def Encoding_LogC4(x):
    a = (np.power(2.0, 18.0) - 16.0) / 117.45
    b = (1023.0 - 95.0) / 1023.0
    c = 95.0 / 1023.0
    s = (7 * np.log(2) * np.power(2.0, 7 - 14 * c / b)) / (a * b)
    t = (np.power(2.0, 14.0 * (-c / b) + 6.0) - 64.0) / a
    return (x < t) * np.divide(np.subtract(x,t),s) + (x >= t) * np.add(np.multiply(np.divide( np.subtract(np.log2(np.add(np.multiply(a,x),64.0)),6.0),14.0),b),c)

def Decoding_LogC4(x):
    a = (np.power(2.0, 18.0) - 16.0) / 117.45
    b = (1023.0 - 95.0) / 1023.0
    c = 95.0 / 1023.0
    s = (7 * np.log(2) * np.power(2.0, 7 - 14 * c / b)) / (a * b)
    t = (np.power(2.0, 14.0 * (-c / b) + 6.0) - 64.0) / a
    return (x < 0.0) * np.add(np.multiply(x,s),t) + (x >= 0.0) * np.divide(np.subtract(np.power(2.0, np.add(np.multiply(14,np.divide(np.subtract(x,c),b)),6)),64.0),a)


def full2legal(x):
    return (219*(x)+16)*np.power(2,(10-8))

def legal2full(x):
    return (((x/np.power(2,(10-8)))-16)/219)

def lin2log2(x):
    return np.log2(x)

def log22lin(x):
    return np.power(2,x)

def Blenden_Differenz(num1, num2):
    new2 = num2 / num1
    return np.log2(new2)

def mse_calculation(imageA, imageB):
    # Expects Luma Images
	err = np.mean(np.square(np.subtract(imageA,imageB)))
	return err

In [ ]:
#@njit
def values_lower_minus_11(array):
    index = (array[:] < 0.006103515625)
    return array[index]

#@njit
def values_minus_11_to_10_5(array):
    index = (0.006103515625 <= array[:,]) & (array[:] < 0.0091552734375)
    return array[index]

#@njit
def values_minus_10_5_to_10(array):
    index = (0.0091552734375 <= array[:,]) & (array[:] < 0.01220703125)
    return array[index]

#@njit
def values_minus_10_to_9_5(array):
    index = (0.01220703125 <= array[:,]) & (array[:] < 0.018310546875)
    return array[index]

#@njit
def values_minus_9_5_to_9(array):
    index = (0.018310546875 <= array[:]) & (array[:] < 0.0244140625)
    return array[index]

#@njit
def values_minus_9_to_8_5(array):
    index = (0.0244140625 <= array[:]) & (array[:] < 0.03662109375)
    return array[index]

#@njit
def values_minus_8_5_to_8(array):
    index = (0.03662109375 <= array[:]) & (array[:] < 0.048828125)
    return array[index]

#@njit
def values_minus_8_to_7_5(array):
    index = (0.048828125 <= array[:]) & (array[:] < 0.0732421875)
    return array[index]

#@njit
def values_minus_7_5_to_7(array):
    index = (0.0732421875 <= array[:]) & (array[:] < 0.09765625)
    return array[index]

#@njit
def values_minus_7_to_6_5(array):
    index = (0.09765625 <= array[:]) & (array[:] < 0.146484375)
    return array[index]

#@njit
def values_minus_6_5_to_6(array):
    index = (0.146484375 <= array[:]) & (array[:] < 0.1953125)
    return array[index]

#@njit
def values_minus_6_to_5_5(array):
    index = (0.1953125 <= array[:]) & (array[:] < 0.29296875)
    return array[index]

#@njit
def values_minus_5_5_to_5(array):
    index = (0.29296875 <= array[:]) & (array[:] < 0.390625)
    return array[index]

#@njit
def values_minus_5_to_4_5(array):
    index = (0.390625 <= array[:]) & (array[:] < 0.5859375)
    return array[index]

#@njit
def values_minus_4_5_to_4(array):
    index = (0.5859375 <= array[:]) & (array[:] < 0.78125)
    return array[index]

#@njit
def values_minus_4_to_3_5(array):
    index = (0.78125 <= array[:]) & (array[:] < 1.171875)
    return array[index]

#@njit
def values_minus_3_5_to_3(array):
    index = (1.171875 <= array[:]) & (array[:] < 1.5625)
    return array[index]

#@njit
def values_minus_3_to_2_5(array):
    index = (1.5625 <= array[:]) & (array[:] < 2.34375)
    return array[index]

#@njit
def values_minus_2_5_to_2(array):
    index = (2.34375 <= array[:]) & (array[:] < 3.125)
    return array[index]

#@njit
def values_minus_2_to_1_5(array):
    index = (3.125 <= array[:]) & (array[:] < 4.6875)
    return array[index]

#@njit
def values_minus_1_5_to_1(array):
    index = (4.6875 <= array[:]) & (array[:] < 6.25)
    return array[index]

#@njit
def values_minus_1_to_0_5(array):
    index = (6.25 <= array[:]) & (array[:] < 9.375)
    return array[index]

#@njit
def values_minus_0_5_to_0(array):
    index = (9.375 <= array[:]) & (array[:] < 12.5)
    return array[index]

#@njit
def values_plus_0_to_0_5(array):
    index = (12.5 <= array[:]) & (array[:] < 18.75)
    return array[index]

#@njit
def values_plus_0_5_to_1(array):
    index = (18.75 <= array[:]) & (array[:] < 25)
    return array[index]

#@njit
def values_plus_1_to_1_5(array):
    index = (25 <= array[:]) & (array[:] < 37.5)
    return array[index]

#@njit
def values_plus_1_5_to_2(array):
    index = (37.5 <= array[:]) & (array[:] < 50)
    return array[index]

#@njit
def values_plus_2_to_2_5(array):
    index = (50 <= array[:]) & (array[:] < 75)
    return array[index]

#@njit
def values_plus_2_5_to_3(array):
    index = (75 <= array[:]) & (array[:] < 100)
    return array[index]

#@njit
def values_plus_3_to_3_5(array):
    index = (100 <= array[:]) & (array[:] < 150)
    return array[index]

#@njit
def values_plus_3_5_to_4(array):
    index = (150 <= array[:]) & (array[:] < 200)
    return array[index]

#@njit
def values_plus_4_to_4_5(array):
    index = (200 <= array[:]) & (array[:] < 300)
    return array[index]

#@njit
def values_plus_4_5_to_5(array):
    index = (300 <= array[:]) & (array[:] < 400)
    return array[index]

#@njit
def values_plus_5_to_5_5(array):
    index = (400 <= array[:]) & (array[:] < 600)
    return array[index]

#@njit
def values_plus_5_5_to_6(array):
    index = (600 <= array[:]) & (array[:] < 800)
    return array[index]

#@njit
def values_over_6(array):
    index = (800 <= array[:])
    return array[index]

In [ ]:
## Log C4 Encoding


def split_array_to_stops(input):

    value_counter_image = np.zeros((36))

    global_values = input
    
    value_counter_image = [len(values_lower_minus_11(global_values)), 
                           len(values_minus_11_to_10_5(global_values)),
                           len(values_minus_10_5_to_10(global_values)),
                           len(values_minus_10_to_9_5(global_values)),
                           len(values_minus_9_5_to_9(global_values)),
                           len(values_minus_9_to_8_5(global_values)),
                           len(values_minus_8_5_to_8(global_values)),
                           len(values_minus_8_to_7_5(global_values)),
                           len(values_minus_7_5_to_7(global_values)),
                           len(values_minus_7_to_6_5(global_values)),
                           len(values_minus_6_5_to_6(global_values)),
                           len(values_minus_6_to_5_5(global_values)),
                           len(values_minus_5_5_to_5(global_values)),
                           len(values_minus_5_to_4_5(global_values)),
                           len(values_minus_4_5_to_4(global_values)),
                           len(values_minus_4_to_3_5(global_values)),
                           len(values_minus_3_5_to_3(global_values)),
                           len(values_minus_3_to_2_5(global_values)),
                           len(values_minus_2_5_to_2(global_values)),
                           len(values_minus_2_to_1_5(global_values)),
                           len(values_minus_1_5_to_1(global_values)),
                           len(values_minus_1_to_0_5(global_values)),
                           len(values_minus_0_5_to_0(global_values)),
                           len(values_plus_0_to_0_5(global_values)),
                           len(values_plus_0_5_to_1(global_values)),
                           len(values_plus_1_to_1_5(global_values)),
                           len(values_plus_1_5_to_2(global_values)),
                           len(values_plus_2_to_2_5(global_values)),
                           len(values_plus_2_5_to_3(global_values)),
                           len(values_plus_3_to_3_5(global_values)),
                           len(values_plus_3_5_to_4(global_values)),
                           len(values_plus_4_to_4_5(global_values)),
                           len(values_plus_4_5_to_5(global_values)),
                           len(values_plus_5_to_5_5(global_values)),
                           len(values_plus_5_5_to_6(global_values)),
                           len(values_over_6(global_values))]
                           
    return value_counter_image

def image_analysis(file):

    axI_Scene_Linear = file

    axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1) * 100

    value_counter_image_r = split_array_to_stops(axI_Scene_Linear_Line_r[:,0])
    value_counter_image_g = split_array_to_stops(axI_Scene_Linear_Line_g[:,0])
    value_counter_image_b = split_array_to_stops(axI_Scene_Linear_Line_b[:,0])

    return np.dstack((value_counter_image_r,value_counter_image_g,value_counter_image_b))


In [ ]:
#@njit
def values_lower_minus_11(array):
    index = (array[:] < 0.006103515625)
    return array[index]

#@njit
def values_minus_11_to_10_5(array):
    index = (0.006103515625 <= array[:,]) & (array[:] < 0.01220703125)
    return array[index]

#@njit
def values_minus_10_to_9_5(array):
    index = (0.01220703125 <= array[:,]) & (array[:] < 0.0244140625)
    return array[index]

#@njit
def values_minus_9_to_8_5(array):
    index = (0.0244140625 <= array[:]) & (array[:] < 0.048828125)
    return array[index]

#@njit
def values_minus_8_to_7_5(array):
    index = (0.048828125 <= array[:]) & (array[:] < 0.09765625)
    return array[index]

#@njit
def values_minus_7_to_6_5(array):
    index = (0.09765625 <= array[:]) & (array[:] < 0.1953125)
    return array[index]

#@njit
def values_minus_6_to_5_5(array):
    index = (0.1953125 <= array[:]) & (array[:] < 0.390625)
    return array[index]

#@njit
def values_minus_5_to_4_5(array):
    index = (0.390625 <= array[:]) & (array[:] < 0.78125)
    return array[index]

#@njit
def values_minus_4_to_3_5(array):
    index = (0.78125 <= array[:]) & (array[:] < 1.5625)
    return array[index]

#@njit
def values_minus_3_to_2_5(array):
    index = (1.5625 <= array[:]) & (array[:] < 3.125)
    return array[index]

#@njit
def values_minus_2_to_1_5(array):
    index = (3.125 <= array[:]) & (array[:] < 6.25)
    return array[index]

#@njit
def values_minus_1_to_0_5(array):
    index = (6.25 <= array[:]) & (array[:] < 12.5)
    return array[index]

#@njit
def values_plus_0_to_0_5(array):
    index = (12.5 <= array[:]) & (array[:] < 25)
    return array[index]

#@njit
def values_plus_1_to_1_5(array):
    index = (25 <= array[:]) & (array[:] < 50)
    return array[index]

#@njit
def values_plus_2_to_2_5(array):
    index = (50 <= array[:]) & (array[:] < 100)
    return array[index]

#@njit
def values_plus_3_to_3_5(array):
    index = (100 <= array[:]) & (array[:] < 200)
    return array[index]

#@njit
def values_plus_4_to_4_5(array):
    index = (200 <= array[:]) & (array[:] < 400)
    return array[index]

#@njit
def values_plus_5_to_5_5(array):
    index = (400 <= array[:]) & (array[:] < 800)
    return array[index]

#@njit
def values_over_6(array):
    index = (800 <= array[:])
    return array[index]

In [ ]:
def split_array_to_stops(input):

    value_counter_image = np.zeros((36))

    global_values = input
    
    value_counter_image = [len(values_lower_minus_11(global_values)), 
                           len(values_minus_11_to_10_5(global_values)),
                           len(values_minus_10_to_9_5(global_values)),
                           len(values_minus_9_to_8_5(global_values)),
                           len(values_minus_8_to_7_5(global_values)),
                           len(values_minus_7_to_6_5(global_values)),
                           len(values_minus_6_to_5_5(global_values)),
                           len(values_minus_5_to_4_5(global_values)),
                           len(values_minus_4_to_3_5(global_values)),
                           len(values_minus_3_to_2_5(global_values)),
                           len(values_minus_2_to_1_5(global_values)),
                           len(values_minus_1_to_0_5(global_values)),
                           len(values_plus_0_to_0_5(global_values)),
                           len(values_plus_1_to_1_5(global_values)),
                           len(values_plus_2_to_2_5(global_values)),
                           len(values_plus_3_to_3_5(global_values)),
                           len(values_plus_4_to_4_5(global_values)),
                           len(values_plus_5_to_5_5(global_values)),
                           len(values_over_6(global_values))]
                           
    return value_counter_image

def image_analysis(file):

    axI_Scene_Linear = file

    axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1) * 100

    value_counter_image_r = split_array_to_stops(axI_Scene_Linear_Line_r[:,0])
    value_counter_image_g = split_array_to_stops(axI_Scene_Linear_Line_g[:,0])
    value_counter_image_b = split_array_to_stops(axI_Scene_Linear_Line_b[:,0])

    return np.dstack((value_counter_image_r,value_counter_image_g,value_counter_image_b))


In [ ]:
value_counter_image_hlg_FR = split_array_to_stops(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0.005,1000))
value_counter_image_pq_FR = split_array_to_stops(c.models.eotf_ST2084(legal2full(np.linspace(78,722,645))))

In [ ]:
root_directory = '/Volumes/T7_Shield/XX_WATCHFOLDER_PQ_HLG_SUMMARY'

big_array = np.zeros((80,1,36))
big_array_names = []
counter = 0

for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith('.csv'):
            csv_file_path = os.path.join(root, file)
            print(file)

            rows = []
            columns = []

            with open(csv_file_path, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                
                columns = next(csvreader)
                
                for row in csvreader:
                    rows.append(row)

            big_array[counter,0] = rows[3]
            big_array_names.append(f"{file}")

            print("Read CSV Line")
            counter = counter + 1

big_array_sum = big_array

In [ ]:
my_array = np.random.randint(1, 10, size=(80, 1, 36))

big_array_min = np.zeros((80, 1, 36))
big_array_mid = np.zeros((80, 1, 36))
big_array_max = np.zeros((80, 1, 36))

names_min = []
names_mid = []
names_max = []

# Calculate the sum along the third axis
for i in range(80):
    sums1 = np.sum(big_array_sum[i,0,0:16])
    sums2 = np.sum(big_array_sum[i,0,16:26])
    sums3 = np.sum(big_array_sum[i,0,26:36])
    print(sums1)
    print(sums2)
    print(sums3)

    if sums1 > 0.4:
        big_array_min[i,0,:] = big_array_sum[i,0,:]
        names_min.append(big_array_names[i])
    if sums2 > 0.4:
        big_array_mid[i,0,:] = big_array_sum[i,0,:]
        names_mid.append(big_array_names[i])
    if sums3 > 0.4:
        big_array_max[i,0,:] = big_array_sum[i,0,:]
        names_max.append(big_array_names[i])

In [ ]:
big_array_all_values = np.zeros((1,1,36))
for i in range (80):
    for x in range(36):
        big_array_all_values[0,0,x] = big_array_all_values[0,0,x] + big_array[i,0,x]

In [ ]:
big_array_all_values_min = np.zeros((1,1,36))
for i in range (114):
    for x in range(36):
        big_array_all_values_min[0,0,x] = big_array_all_values_min[0,0,x] + big_array_min[i,0,x]

In [ ]:
big_array_all_values_mid = np.zeros((1,1,36))
for i in range (114):
    for x in range(36):
        big_array_all_values_mid[0,0,x] = big_array_all_values_mid[0,0,x] + big_array_mid[i,0,x]

In [ ]:
big_array_all_values_max = np.zeros((1,1,36))
for i in range (114):
    for x in range(36):
        big_array_all_values_max[0,0,x] = big_array_all_values_max[0,0,x] + big_array_max[i,0,x]

In [ ]:
for i in range(len(big_array_sum[:,0,0])):
    big_array_sum[i,0,:] = np.divide(big_array_sum[i,0,:],sum(big_array_sum[i,0,:]))

In [ ]:
value_counter_image_hlg_FR = big_array_all_values

In [ ]:
value_counter_image_pq_FR = big_array_all_values

In [ ]:
value_counter_image_hlg_FR_percentage = np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR))
value_counter_image_pq_FR_percentage = np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR))

In [ ]:
big_array_auswertung = np.full((30,1,36), "Hello")

In [ ]:
for i in range(len(big_array_auswertung[:,0,0])):
    for x in range(36):
        diff_hlg = np.subtract(value_counter_image_hlg_FR_percentage[x],big_array_sum[i,0,x])
        diff_pq = np.subtract(value_counter_image_pq_FR_percentage[x],big_array_sum[i,0,x])
        if (diff_hlg >= 0) & (diff_pq >= 0) & (abs(diff_hlg) > (abs(diff_pq))):
            big_array_auswertung[i,0,x] = "PQ"
        if (diff_hlg >= 0) & (diff_pq >= 0) & (abs(diff_pq) > (abs(diff_hlg))):
            big_array_auswertung[i,0,x] = "HLG"
        if (diff_hlg <= 0) & (diff_pq <= 0) & (abs(diff_hlg) > (abs(diff_pq))):
            big_array_auswertung[i,0,x] = "PQ"
        if (diff_hlg <= 0) & (diff_pq <= 0) & (abs(diff_pq) > (abs(diff_hlg))):
            big_array_auswertung[i,0,x] = "HLG"

        if (diff_hlg >= 0) & (diff_pq <= 0):
            big_array_auswertung[i,0,x] = "HLG"
        if (diff_hlg <= 0) & (diff_pq >= 0):
            big_array_auswertung[i,0,x] = "PQ"


In [ ]:
big_array_auswertung_min = np.full((114,1,36), "Hello")
for i in range(len(big_array_auswertung_min[:,0,0])):
    for x in range(36):
        diff_hlg = np.subtract(value_counter_image_hlg_FR_percentage[x],big_array_min[i,0,x])
        diff_pq = np.subtract(value_counter_image_pq_FR_percentage[x],big_array_min[i,0,x])
        if (diff_hlg != value_counter_image_hlg_FR_percentage[x]) & (diff_pq != value_counter_image_pq_FR_percentage[x]):
            if (diff_hlg >= 0) & (diff_pq >= 0) & (abs(diff_hlg) > (abs(diff_pq))):
                big_array_auswertung_min[i,0,x] = "PQ"
            if (diff_hlg >= 0) & (diff_pq >= 0) & (abs(diff_pq) > (abs(diff_hlg))):
                big_array_auswertung_min[i,0,x] = "HLG"
            if (diff_hlg <= 0) & (diff_pq <= 0) & (abs(diff_hlg) > (abs(diff_pq))):
                big_array_auswertung_min[i,0,x] = "PQ"
            if (diff_hlg <= 0) & (diff_pq <= 0) & (abs(diff_pq) > (abs(diff_hlg))):
                big_array_auswertung_min[i,0,x] = "HLG"

            if (diff_hlg >= 0) & (diff_pq <= 0):
                big_array_auswertung_min[i,0,x] = "HLG"
            if (diff_hlg <= 0) & (diff_pq >= 0):
                big_array_auswertung_min[i,0,x] = "PQ"


In [ ]:
big_array_auswertung_mid = np.full((114,1,36), "Hello")
for i in range(len(big_array_auswertung_mid[:,0,0])):
    for x in range(36):
        diff_hlg = np.subtract(value_counter_image_hlg_FR_percentage[x],big_array_mid[i,0,x])
        diff_pq = np.subtract(value_counter_image_pq_FR_percentage[x],big_array_mid[i,0,x])
        if (diff_hlg != value_counter_image_hlg_FR_percentage[x]) & (diff_pq != value_counter_image_pq_FR_percentage[x]):
            if (diff_hlg >= 0) & (diff_pq >= 0) & (abs(diff_hlg) > (abs(diff_pq))):
                big_array_auswertung_mid[i,0,x] = "PQ"
            if (diff_hlg >= 0) & (diff_pq >= 0) & (abs(diff_pq) > (abs(diff_hlg))):
                big_array_auswertung_mid[i,0,x] = "HLG"
            if (diff_hlg <= 0) & (diff_pq <= 0) & (abs(diff_hlg) > (abs(diff_pq))):
                big_array_auswertung_mid[i,0,x] = "PQ"
            if (diff_hlg <= 0) & (diff_pq <= 0) & (abs(diff_pq) > (abs(diff_hlg))):
                big_array_auswertung_mid[i,0,x] = "HLG"

            if (diff_hlg >= 0) & (diff_pq <= 0):
                big_array_auswertung_mid[i,0,x] = "HLG"
            if (diff_hlg <= 0) & (diff_pq >= 0):
                big_array_auswertung_mid[i,0,x] = "PQ"

In [ ]:
big_array_auswertung_max = np.full((114,1,36), "Hello")
for i in range(len(big_array_auswertung_max[:,0,0])):
    for x in range(36):
        diff_hlg = np.subtract(value_counter_image_hlg_FR_percentage[x],big_array_max[i,0,x])
        diff_pq = np.subtract(value_counter_image_pq_FR_percentage[x],big_array_max[i,0,x])
        if (diff_hlg != value_counter_image_hlg_FR_percentage[x]) & (diff_pq != value_counter_image_pq_FR_percentage[x]):
            if (diff_hlg >= 0) & (diff_pq >= 0) & (abs(diff_hlg) > (abs(diff_pq))):
                big_array_auswertung_max[i,0,x] = "PQ"
            if (diff_hlg >= 0) & (diff_pq >= 0) & (abs(diff_pq) > (abs(diff_hlg))):
                big_array_auswertung_max[i,0,x] = "HLG"
            if (diff_hlg <= 0) & (diff_pq <= 0) & (abs(diff_hlg) > (abs(diff_pq))):
                big_array_auswertung_max[i,0,x] = "PQ"
            if (diff_hlg <= 0) & (diff_pq <= 0) & (abs(diff_pq) > (abs(diff_hlg))):
                big_array_auswertung_max[i,0,x] = "HLG"

            if (diff_hlg >= 0) & (diff_pq <= 0):
                big_array_auswertung_max[i,0,x] = "HLG"
            if (diff_hlg <= 0) & (diff_pq >= 0):
                big_array_auswertung_max[i,0,x] = "PQ"

In [ ]:
import csv

def write_to_csv(file_path, array1, array2):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Iterate through both arrays and write to CSV

        for i in range(len(big_array_auswertung[:,0,0])): 
                row_data = [array1[i]] + [array2[i,0,:]]
                writer.writerow(row_data)

write_to_csv('/Volumes/T7_Shield/71_AUSWERTUNG_SCENE_LINEAR/output.csv', big_array_names, big_array_auswertung)


In [ ]:
for i in range(114):
    for x in range(36):
        if big_array_auswertung[i,0,x] == "PQ":
            hlg_pq_counter[0,x,0] = hlg_pq_counter[0,x,0] + 1
        if big_array_auswertung[i,0,x] == "HLG":
            hlg_pq_counter[0,x,1] = hlg_pq_counter[0,x,1] + 1

In [ ]:
hlg_pq_counter_min = np.zeros((1,36,2))
hlg_pq_counter_mid = np.zeros((1,36,2))
hlg_pq_counter_max = np.zeros((1,36,2))

for i in range(114):
    for x in range(36):
        if big_array_auswertung_min[i,0,x] == "PQ":
            hlg_pq_counter_min[0,x,0] = hlg_pq_counter_min[0,x,0] + 1
        if big_array_auswertung_min[i,0,x] == "HLG":
            hlg_pq_counter_min[0,x,1] = hlg_pq_counter_min[0,x,1] + 1
        if big_array_auswertung_min[i,0,x] == "Hello":
            hlg_pq_counter_min[0,x,1] = hlg_pq_counter_min[0,x,1] + 0  

for i in range(114):
    for x in range(36):
        if big_array_auswertung_mid[i,0,x] == "PQ":
            hlg_pq_counter_mid[0,x,0] = hlg_pq_counter_mid[0,x,0] + 1
        if big_array_auswertung_mid[i,0,x] == "HLG":
            hlg_pq_counter_mid[0,x,1] = hlg_pq_counter_mid[0,x,1] + 1
        if big_array_auswertung_mid[i,0,x] == "Hello":
            hlg_pq_counter_min[0,x,1] = hlg_pq_counter_min[0,x,1] + 0  

for i in range(114):
    for x in range(36):
        if big_array_auswertung_max[i,0,x] == "PQ":
            hlg_pq_counter_max[0,x,0] = hlg_pq_counter_max[0,x,0] + 1
        if big_array_auswertung_max[i,0,x] == "HLG":
            hlg_pq_counter_max[0,x,1] = hlg_pq_counter_max[0,x,1] + 1
        if big_array_auswertung_mid[i,0,x] == "Hello":
            hlg_pq_counter_min[0,x,1] = hlg_pq_counter_min[0,x,1] + 0  

In [ ]:
hlg_pq_counter_percentage = np.zeros((1,36,2))
for i in range(36):
    hlg_pq_counter_percentage[0,i,0] = np.divide(hlg_pq_counter[0,i,0],114)
    hlg_pq_counter_percentage[0,i,1] = np.divide(hlg_pq_counter[0,i,1],114)

In [ ]:
hlg_pq_counter_percentage_min = np.zeros((1,36,2))
for i in range(36):
    hlg_pq_counter_percentage_min[0,i,0] = np.divide(hlg_pq_counter_min[0,i,0],21)
    hlg_pq_counter_percentage_min[0,i,1] = np.divide(hlg_pq_counter_min[0,i,1],21)

hlg_pq_counter_percentage_mid = np.zeros((1,36,2))
for i in range(36):
    hlg_pq_counter_percentage_mid[0,i,0] = np.divide(hlg_pq_counter_mid[0,i,0],61)
    hlg_pq_counter_percentage_mid[0,i,1] = np.divide(hlg_pq_counter_mid[0,i,1],61)

hlg_pq_counter_percentage_max = np.zeros((1,36,2))
for i in range(36):
    hlg_pq_counter_percentage_max[0,i,0] = np.divide(hlg_pq_counter_max[0,i,0],48)
    hlg_pq_counter_percentage_max[0,i,1] = np.divide(hlg_pq_counter_max[0,i,1],48)

In [ ]:
figR, (axL1, axL) = plt.subplots(2, 1, figsize=(16, 10), constrained_layout=True, gridspec_kw={'height_ratios': [1 , 1]})


value_counter_image_hlg_FR = np.zeros((36))
value_counter_image_pq_FR = np.zeros((36))

axL1.set_ylabel('Amount of Code Values Used [%]')
axL1.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL1.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL1.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)



axL1.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)


value_counter_image_hlg_FR = split_array_to_stops(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0,1000))
value_counter_image_pq_FR = split_array_to_stops(c.models.eotf_ST2084(legal2full(np.linspace(64,722,659))))


bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

axL1.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=2,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=1)
axL1.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=2,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=1)

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL1.set_xticks(y_pos_neu+1.5, y_pos_between)

axL1.set_xlim(0.5,19.5)
axL1.set_yscale("log")
axL1.set_ylim(0.0001,1)
axL1.set_yticks([0.0001,0.001,0.01,0.1,1],[0.01,0.1,1,10,100])


axL.set_ylabel('Preferred Image Encoding Function [%]')
axL.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)

axL.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)


# value_counter_image_hlg_FR = split_array_to_stops(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0,1000))
# value_counter_image_pq_FR = split_array_to_stops(c.models.eotf_ST2084(legal2full(np.linspace(64,940,877))))


bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)
axL.step(y_pos/2 + 1.25, hlg_pq_counter_percentage[0,:,0], 'k',where="mid",linewidth=3,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=1)
axL.step(y_pos/2 + 1.25, hlg_pq_counter_percentage[0,:,1], 'k',where="mid",linewidth=3,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=1)

counter1 = 1
counter2 = 1
for x, y in zip(y_pos[1:35]/2 + 1.25, hlg_pq_counter_percentage[0,1:35,0]):
    axL.annotate(int(np.round(hlg_pq_counter_percentage[0,counter1,0]*100,0)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=12,color="darkgreen")
    counter1 = counter1 + 1

axL.annotate(int(np.round(hlg_pq_counter_percentage[0,0,0]*100,0)), (y_pos[0:1]/2 + 1.25, hlg_pq_counter_percentage[0,0:1,0]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkgreen")
axL.annotate(int(np.round(hlg_pq_counter_percentage[0,35,0]*100,0)), (y_pos[35:36]/2 + 1.25, hlg_pq_counter_percentage[0,35:36,0]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkgreen")


for x, y in zip(y_pos[1:35]/2 + 1.25, hlg_pq_counter_percentage[0,1:35,1]):
    axL.annotate(int(np.round(hlg_pq_counter_percentage[0,counter2,1]*100,0)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=12,color="darkturquoise")
    counter2 = counter2 + 1

axL.annotate(int(np.round(hlg_pq_counter_percentage[0,0,1]*100,0)), (y_pos[0:1]/2 + 1.25, hlg_pq_counter_percentage[0,0:1,1]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkturquoise")
axL.annotate(int(np.round(hlg_pq_counter_percentage[0,35,1]*100,0)), (y_pos[35:36]/2 + 1.25, hlg_pq_counter_percentage[0,35:36,1]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkturquoise")


# for x, y in zip(y_pos/2 + 1.25, hlg_pq_counter_percentage[0,:,1]):
#     axL.annotate(f'{y}', (x, y), textcoords="offset points", xytext=(0,0), ha='center')

print(hlg_pq_counter_percentage[0,:,1])

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL.set_xticks(y_pos_neu+1.5, y_pos_between)

axL.set_xlim(0.5,19.5)
axL.set_ylim(0,1)
axL.set_yticks([0,0.25,0.5,0.75,1],[0,25,50,75,100])


axL.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))

axL1.set_title("Code Value Distribution - Display Linear Light // Luma")
axL.set_title("Preffered Encoding Function // 114 Stills")

axL.text(0.88,0.5,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL1.text(0.88,0.01,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL.text(19.16,0.5,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL1.text(19.16,0.01,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)


In [ ]:
figR, (axL, axL1) = plt.subplots(2, 1, figsize=(16, 10), constrained_layout=True, gridspec_kw={'height_ratios': [1 , 1]})


value_counter_image_hlg_FR = np.zeros((36))
value_counter_image_pq_FR = np.zeros((36))

axL.set_ylabel('Preferred Image Encoding Function [%]')
axL.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)

axL.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)



bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

folder = big_array_all_values_min[0,0,:]/sum(big_array_all_values_min[0,0,:])

# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)
axL.step(y_pos/2 + 1.25, big_array_all_values_min[0,0,:]/sum(big_array_all_values_min[0,0,:]), 'k',where="mid",linewidth=3,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=1)

counter1 = 1
counter2 = 1
for x, y in zip(y_pos[1:35]/2 + 1.25, folder[1:35]):
    axL.annotate((np.round(folder[counter1]*100,1)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=14,color="darkgreen")
    counter1 = counter1 + 1

axL.annotate(int(np.round(big_array_all_values_min[0,0,0])), (y_pos[0:1]/2 + 1.25, big_array_all_values_min[0,0,0:1]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkgreen")
axL.annotate(int(np.round(big_array_all_values_min[0,0,35])), (y_pos[35:36]/2 + 1.25, big_array_all_values_min[0,0,35:36]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkgreen")


print(hlg_pq_counter_percentage[0,:,1])

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL.set_xticks(y_pos_neu+1.5, y_pos_between)

axL.set_xlim(0.5,19.5)
axL.set_yscale("log")
axL.set_ylim(0.0001,1)
axL.set_yticks([0.0001,0.001,0.01,0.1,1],[0.01,0.1,1,10,100])



axL.set_title("Content Distribution Display Linear Light // 114 Stills")


axL1.set_ylabel('Preferred Image Encoding Function [%]')
axL1.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL1.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL1.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)

axL1.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)



bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)
axL1.step(y_pos/2 + 1.25, hlg_pq_counter_percentage_min[0,:,0], 'k',where="mid",linewidth=3,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=1)
axL1.step(y_pos/2 + 1.25, hlg_pq_counter_percentage_min[0,:,1], 'k',where="mid",linewidth=3,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=1)

counter1 = 1
counter2 = 1
for x, y in zip(y_pos[1:35]/2 + 1.25, hlg_pq_counter_percentage_min[0,1:35,0]):
    axL1.annotate(int(np.round(hlg_pq_counter_percentage_min[0,counter1,0]*100,0)), (x, y), textcoords="offset points", xytext=(0,-15), ha='center',fontsize=12,color="darkgreen")
    counter1 = counter1 + 1

axL1.annotate(int(np.round(hlg_pq_counter_percentage_min[0,0,0]*100,0)), (y_pos[0:1]/2 + 1.25, hlg_pq_counter_percentage_min[0,0:1,0]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkgreen")
axL1.annotate(int(np.round(hlg_pq_counter_percentage_min[0,35,0]*100,0)), (y_pos[35:36]/2 + 1.25, hlg_pq_counter_percentage_min[0,35:36,0]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkgreen")


for x, y in zip(y_pos[1:35]/2 + 1.25, hlg_pq_counter_percentage_min[0,1:35,1]):
    axL1.annotate(int(np.round(hlg_pq_counter_percentage_min[0,counter2,1]*100,0)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=12,color="darkturquoise")
    counter2 = counter2 + 1

axL1.annotate(int(np.round(hlg_pq_counter_percentage_min[0,0,1]*100,0)), (y_pos[0:1]/2 + 1.25, hlg_pq_counter_percentage_min[0,0:1,1]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkturquoise")
axL1.annotate(int(np.round(hlg_pq_counter_percentage_min[0,35,1]*100,0)), (y_pos[35:36]/2 + 1.25, hlg_pq_counter_percentage_min[0,35:36,1]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkturquoise")


# for x, y in zip(y_pos/2 + 1.25, hlg_pq_counter_percentage[0,:,1]):
#     axL.annotate(f'{y}', (x, y), textcoords="offset points", xytext=(0,0), ha='center')

print(hlg_pq_counter_percentage_min[0,:,1])

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL1.set_xticks(y_pos_neu+1.5, y_pos_between)

axL1.set_xlim(0.5,19.5)
axL1.set_ylim(0,1)
axL1.set_yticks([0,0.25,0.5,0.75,1],[0,25,50,75,100])

axL.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))

axL1.set_title("Code Value Distribution - Display Linear Light // Luma")
axL.set_title("Preffered Encoding Function // 21 Low-Key Stills")

axL.text(0.88,0.01,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL1.text(0.88,0.5,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL.text(19.16,0.01,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL1.text(19.16,0.5,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)


In [ ]:
figR, (axL, axL1) = plt.subplots(2, 1, figsize=(16, 10), constrained_layout=True, gridspec_kw={'height_ratios': [1 , 1]})


value_counter_image_hlg_FR = np.zeros((36))
value_counter_image_pq_FR = np.zeros((36))

axL.set_ylabel('Preferred Image Encoding Function [%]')
axL.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)

axL.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)




bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

folder = big_array_all_values_mid[0,0,:]/sum(big_array_all_values_mid[0,0,:])

# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)
axL.step(y_pos/2 + 1.25, big_array_all_values_mid[0,0,:]/sum(big_array_all_values_mid[0,0,:]), 'k',where="mid",linewidth=3,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=1)

counter1 = 1
counter2 = 1
for x, y in zip(y_pos[1:35]/2 + 1.25, folder[1:35]):
    axL.annotate((np.round(folder[counter1]*100,1)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=14,color="darkgreen")
    counter1 = counter1 + 1

axL.annotate(int(np.round(big_array_all_values_mid[0,0,0])), (y_pos[0:1]/2 + 1.25, big_array_all_values_mid[0,0,0:1]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkgreen")
axL.annotate(int(np.round(big_array_all_values_mid[0,0,35])), (y_pos[35:36]/2 + 1.25, big_array_all_values_mid[0,0,35:36]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkgreen")


print(hlg_pq_counter_percentage[0,:,1])

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL.set_xticks(y_pos_neu+1.5, y_pos_between)

axL.set_xlim(0.5,19.5)
axL.set_yscale("log")
axL.set_ylim(0.0001,1)
axL.set_yticks([0.0001,0.001,0.01,0.1,1],[0.01,0.1,1,10,100])



axL.set_title("Content Distribution Display Linear Light // 114 Stills")


axL1.set_ylabel('Preferred Image Encoding Function [%]')
axL1.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL1.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL1.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)

axL1.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)


# value_counter_image_hlg_FR = split_array_to_stops(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0,1000))
# value_counter_image_pq_FR = split_array_to_stops(c.models.eotf_ST2084(legal2full(np.linspace(64,940,877))))


bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)
axL1.step(y_pos/2 + 1.25, hlg_pq_counter_percentage_mid[0,:,0], 'k',where="mid",linewidth=3,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=1)
axL1.step(y_pos/2 + 1.25, hlg_pq_counter_percentage_mid[0,:,1], 'k',where="mid",linewidth=3,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=1)

counter1 = 1
counter2 = 1
for x, y in zip(y_pos[1:35]/2 + 1.25, hlg_pq_counter_percentage_mid[0,1:35,0]):
    axL1.annotate(int(np.round(hlg_pq_counter_percentage_mid[0,counter1,0]*100,0)), (x, y), textcoords="offset points", xytext=(0,-15), ha='center',fontsize=12,color="darkgreen")
    counter1 = counter1 + 1

axL1.annotate(int(np.round(hlg_pq_counter_percentage_mid[0,0,0]*100,0)), (y_pos[0:1]/2 + 1.25, hlg_pq_counter_percentage_mid[0,0:1,0]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkgreen")
axL1.annotate(int(np.round(hlg_pq_counter_percentage_mid[0,35,0]*100,0)), (y_pos[35:36]/2 + 1.25, hlg_pq_counter_percentage_mid[0,35:36,0]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkgreen")


for x, y in zip(y_pos[1:35]/2 + 1.25, hlg_pq_counter_percentage_mid[0,1:35,1]):
    axL1.annotate(int(np.round(hlg_pq_counter_percentage_mid[0,counter2,1]*100,0)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=12,color="darkturquoise")
    counter2 = counter2 + 1

axL1.annotate(int(np.round(hlg_pq_counter_percentage_mid[0,0,1]*100,0)), (y_pos[0:1]/2 + 1.25, hlg_pq_counter_percentage_mid[0,0:1,1]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkturquoise")
axL1.annotate(int(np.round(hlg_pq_counter_percentage_mid[0,35,1]*100,0)), (y_pos[35:36]/2 + 1.25, hlg_pq_counter_percentage_mid[0,35:36,1]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkturquoise")


# for x, y in zip(y_pos/2 + 1.25, hlg_pq_counter_percentage[0,:,1]):
#     axL.annotate(f'{y}', (x, y), textcoords="offset points", xytext=(0,0), ha='center')

print(hlg_pq_counter_percentage_mid[0,:,1])

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL1.set_xticks(y_pos_neu+1.5, y_pos_between)

axL1.set_xlim(0.5,19.5)
axL1.set_ylim(0,1)
axL1.set_yticks([0,0.25,0.5,0.75,1],[0,25,50,75,100])

axL.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))

axL1.set_title("Code Value Distribution - Display Linear Light // Luma")
axL.set_title("Preffered Encoding Function // 21 Mid-Key Stills")

axL.text(0.88,0.01,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL1.text(0.88,0.5,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL.text(19.16,0.01,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL1.text(19.16,0.5,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)


In [ ]:
figR, (axL, axL1) = plt.subplots(2, 1, figsize=(16, 10), constrained_layout=True, gridspec_kw={'height_ratios': [1 , 1]})


value_counter_image_hlg_FR = np.zeros((36))
value_counter_image_pq_FR = np.zeros((36))

axL.set_ylabel('Preferred Image Encoding Function [%]')
axL.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)

axL.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)




bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

folder = big_array_all_values_max[0,0,:]/sum(big_array_all_values_max[0,0,:])

# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)
axL.step(y_pos/2 + 1.25, big_array_all_values_max[0,0,:]/sum(big_array_all_values_max[0,0,:]), 'k',where="mid",linewidth=3,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=1)

counter1 = 1
counter2 = 1
for x, y in zip(y_pos[1:35]/2 + 1.25, folder[1:35]):
    axL.annotate((np.round(folder[counter1]*100,1)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=14,color="darkgreen")
    counter1 = counter1 + 1

axL.annotate(int(np.round(big_array_all_values_max[0,0,0])), (y_pos[0:1]/2 + 1.25, big_array_all_values_max[0,0,0:1]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkgreen")
axL.annotate(int(np.round(big_array_all_values_max[0,0,35])), (y_pos[35:36]/2 + 1.25, big_array_all_values_max[0,0,35:36]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkgreen")


print(hlg_pq_counter_percentage[0,:,1])

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL.set_xticks(y_pos_neu+1.5, y_pos_between)

axL.set_xlim(0.5,19.5)
axL.set_yscale("log")
axL.set_ylim(0.0001,1)
axL.set_yticks([0.0001,0.001,0.01,0.1,1],[0.01,0.1,1,10,100])



axL.set_title("Content Distribution Display Linear Light // 114 Stills")


axL1.set_ylabel('Preferred Image Encoding Function [%]')
axL1.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL1.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL1.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)

axL1.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)


# value_counter_image_hlg_FR = split_array_to_stops(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0,1000))
# value_counter_image_pq_FR = split_array_to_stops(c.models.eotf_ST2084(legal2full(np.linspace(64,940,877))))


bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)
axL1.step(y_pos/2 + 1.25, hlg_pq_counter_percentage_max[0,:,0], 'k',where="mid",linewidth=3,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=1)
axL1.step(y_pos/2 + 1.25, hlg_pq_counter_percentage_max[0,:,1], 'k',where="mid",linewidth=3,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=1)

counter1 = 1
counter2 = 1
for x, y in zip(y_pos[1:35]/2 + 1.25, hlg_pq_counter_percentage_max[0,1:35,0]):
    axL1.annotate(int(np.round(hlg_pq_counter_percentage_max[0,counter1,0]*100,0)), (x, y), textcoords="offset points", xytext=(0,-15), ha='center',fontsize=12,color="darkgreen")
    counter1 = counter1 + 1

axL1.annotate(int(np.round(hlg_pq_counter_percentage_max[0,0,0]*100,0)), (y_pos[0:1]/2 + 1.25, hlg_pq_counter_percentage_max[0,0:1,0]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkgreen")
axL1.annotate(int(np.round(hlg_pq_counter_percentage_max[0,35,0]*100,0)), (y_pos[35:36]/2 + 1.25, hlg_pq_counter_percentage_max[0,35:36,0]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkgreen")


for x, y in zip(y_pos[1:35]/2 + 1.25, hlg_pq_counter_percentage_max[0,1:35,1]):
    axL1.annotate(int(np.round(hlg_pq_counter_percentage_max[0,counter2,1]*100,0)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=12,color="darkturquoise")
    counter2 = counter2 + 1

axL1.annotate(int(np.round(hlg_pq_counter_percentage_max[0,0,1]*100,0)), (y_pos[0:1]/2 + 1.25, hlg_pq_counter_percentage_max[0,0:1,1]), textcoords="offset points", xytext=(4,5), ha='center',fontsize=12,color="darkturquoise")
axL1.annotate(int(np.round(hlg_pq_counter_percentage_max[0,35,1]*100,0)), (y_pos[35:36]/2 + 1.25, hlg_pq_counter_percentage_max[0,35:36,1]), textcoords="offset points", xytext=(-7,5), ha='center',fontsize=12,color="darkturquoise")


# for x, y in zip(y_pos/2 + 1.25, hlg_pq_counter_percentage[0,:,1]):
#     axL.annotate(f'{y}', (x, y), textcoords="offset points", xytext=(0,0), ha='center')

print(hlg_pq_counter_percentage_max[0,:,1])

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL1.set_xticks(y_pos_neu+1.5, y_pos_between)

axL1.set_xlim(0.5,19.5)
axL1.set_ylim(0,1)
axL1.set_yticks([0,0.25,0.5,0.75,1],[0,25,50,75,100])

axL.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))

axL1.set_title("Code Value Distribution - Display Linear Light // Luma")
axL.set_title("Preffered Encoding Function // 21 High-Key Stills")

axL.text(0.88,0.01,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL1.text(0.88,0.5,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL.text(19.16,0.01,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
axL1.text(19.16,0.5,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)


In [ ]:
figR, (axL, axL1) = plt.subplots(2, 1, figsize=(16, 10), constrained_layout=True, gridspec_kw={'height_ratios': [1 , 1]})


value_counter_image_hlg_FR = np.zeros((36))
value_counter_image_pq_FR = np.zeros((36))

axL.set_ylabel('Share of total amount in %',fontsize=18)
axL.set_xlabel('Luminance in $\mathregular{cd/m^2}$',fontsize=18)

#x1 = np.linspace(0.000000000000001,1,1000000)

axL.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)

axL.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)




bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A')

y_pos = np.arange(len(bars))

folder = big_array_all_values[0,0,:]/sum(big_array_all_values[0,0,:])
new_array = np.zeros((19))
new_array[0] = folder[0]
new_array[1] = folder[1]+folder[2]
new_array[2] = folder[3]+folder[4]
new_array[3] = folder[5]+folder[6]
new_array[4] = folder[7]+folder[8]
new_array[5] = folder[9]+folder[10]
new_array[6] = folder[11]+folder[12]
new_array[7] = folder[13]+folder[14]
new_array[8] = folder[15]+folder[16]
new_array[9] = folder[17]+folder[18]
new_array[10] = folder[19]+folder[20]
new_array[11] = folder[21]+folder[22]
new_array[12] = folder[23]+folder[24]
new_array[13] = folder[25]+folder[26]
new_array[14] = folder[27]+folder[28]
new_array[15] = folder[29]+folder[30]
new_array[16] = folder[31]+folder[32]
new_array[17] = folder[33]+folder[34]
new_array[18] = folder[35]

# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
# axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)
#axL.step(y_pos + 1, new_array, 'k',where="mid",linewidth=3,color="darkgreen", label="Luminance Distribution 80 stills",alpha=1)

counter1 = 1
counter2 = 1
for x, y in zip(y_pos[1:18] + 1, new_array[1:18]):
    axL.annotate((np.round(new_array[counter1]*100,1)), (x, y), textcoords="offset points", xytext=(0,7), ha='center',fontsize=20,color="black")
    counter1 = counter1 + 1

axL.annotate(np.round(new_array[0]*100,1), (y_pos[0] + 1, new_array[0]), textcoords="offset points", xytext=(30,-21), ha='center',fontsize=20,color="black")
axL.annotate(np.round(new_array[18]*100,1), (y_pos[18] + 1, new_array[18]), textcoords="offset points", xytext=(-30,-21), ha='center',fontsize=20,color="black")



y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL.set_xticks(y_pos_neu+1.5, y_pos_between,fontsize=18)

axL.set_xlim(0.5,19.5)
axL.set_yscale("log")
axL.set_ylim(0.0001,1)
axL.set_yticks([0.0001,0.001,0.01,0.1,1],[0.01,0.1,1,10,100],fontsize=18)


axL.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))

#axL.set_title("Content Distribution Display Linear Light // 114 Stills")

axL.text(0.88,0.01,"0.005 $\mathregular{cd/m^2}$ Limit",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=20)
axL.text(19.16,0.01,"1000 $\mathregular{cd/m^2}$ Limit",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=20)
axL.legend(fontsize=18)

In [ ]:
figR, (axL1) = plt.subplots(1, 1, figsize=(16, 6), constrained_layout=True)


axL1.set_ylabel('Code Values per f-stop in Code Values',fontsize=18)
axL1.set_xlabel('Luminance in $\mathregular{cd/m^2}$',fontsize=18)

#x1 = np.linspace(0.000000000000001,1,1000000)

axL1.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL1.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL1.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL1.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL1.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)



axL1.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL1.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL1.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)


value_counter_image_hlg_FR = split_array_to_stops(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0.005,1000))
value_counter_image_pq_FR = split_array_to_stops(c.models.eotf_ST2084(legal2full(np.linspace(78,722,645))))


bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A')

y_pos = np.arange(len(bars))

axL1.step(y_pos + 1, value_counter_image_hlg_FR, 'k',where="mid",linewidth=2,color="darkturquoise", label="HLG EOTF 10-bit Narrow Range",alpha=1)
axL1.step(y_pos + 1, value_counter_image_pq_FR, 'k',where="mid",linewidth=2,color="darkgreen", label="PQ EOTF 10-bit Narrow Range",alpha=1)

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL1.set_xticks(y_pos_neu+1.5, y_pos_between,fontsize=18)

axL1.set_xlim(0.5,19.5)
axL1.set_ylim(0,125)
axL1.set_yticks([0,25,50,75,100,125],[0,25,50,75,100,125],fontsize=18)

counter1 = 1
counter2 = 1

for x, y in zip(y_pos[1:10] + 1, value_counter_image_pq_FR[1:10]):
    axL1.annotate(int(np.round(value_counter_image_pq_FR[counter1],0)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=20,color="darkgreen")
    counter1 = counter1 + 1

counter1 = 12
for x, y in zip(y_pos[11:18] + 1, value_counter_image_pq_FR[11:18]):
    axL1.annotate(int(np.round(value_counter_image_pq_FR[counter1],0)), (x, y), textcoords="offset points", xytext=(0,5), ha='center',fontsize=20,color="darkgreen")
    counter1 = counter1 + 1


axL1.annotate(int(np.round(value_counter_image_pq_FR[10],0)), (y_pos[10] + 1, value_counter_image_pq_FR[10]), textcoords="offset points", xytext=(0,-20), ha='center',fontsize=20,color="darkgreen")
axL1.annotate(int(np.round(value_counter_image_pq_FR[0],0)), (y_pos[0] + 1, value_counter_image_pq_FR[0]), textcoords="offset points", xytext=(20,25), ha='center',fontsize=20,color="darkgreen")
axL1.annotate(int(np.round(value_counter_image_pq_FR[18],0)), (y_pos[18] + 1, value_counter_image_pq_FR[18]), textcoords="offset points", xytext=(-40,5), ha='center',fontsize=20,color="darkgreen")


for x, y in zip(y_pos[1:10] + 1, value_counter_image_hlg_FR[1:10]):
    axL1.annotate(int(np.round(value_counter_image_hlg_FR[counter2],0)), (x, y), textcoords="offset points", xytext=(0,0-20), ha='center',fontsize=20,color="darkturquoise")
    counter2 = counter2 + 1

counter2 = 12
for x, y in zip(y_pos[11:18] + 1, value_counter_image_hlg_FR[11:18]):
    axL1.annotate(int(np.round(value_counter_image_hlg_FR[counter2],0)), (x, y), textcoords="offset points", xytext=(0,0-20), ha='center',fontsize=20,color="darkturquoise")
    counter2 = counter2 + 1

axL1.annotate(int(np.round(value_counter_image_hlg_FR[10],0)), (y_pos[10] + 1, value_counter_image_hlg_FR[10]), textcoords="offset points", xytext=(0,5), ha='center',fontsize=20,color="darkturquoise")
axL1.annotate(int(np.round(value_counter_image_hlg_FR[0],0)), (y_pos[0] + 1, value_counter_image_hlg_FR[0]), textcoords="offset points", xytext=(20,5), ha='center',fontsize=20,color="darkturquoise")
axL1.annotate(int(np.round(value_counter_image_hlg_FR[18],0)), (y_pos[18] + 1, value_counter_image_hlg_FR[18]), textcoords="offset points", xytext=(-40,5), ha='center',fontsize=20,color="darkturquoise")



axL1.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
axL1.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))

#axL1.set_title("Code Value Distribution - Display Linear Light // Luma")

axL1.text(0.88,50+12.5,"0.005 $\mathregular{cd/m^2}$ Limit",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=20)
axL1.text(19.16,50+12.5,"1000 $\mathregular{cd/m^2}$ Limit",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=20)

axL1.legend(facecolor='white', framealpha=1,loc='upper center',fontsize=16)

In [ ]:
figR, (axL) = plt.subplots(1, 1, figsize=(16, 6), constrained_layout=True)



value_counter_image_hlg_FR = np.zeros((36))
value_counter_image_pq_FR = np.zeros((36))

axL.set_ylabel('Amount of Code Values Used [%]')
axL.set_xlabel('Display Linear Light [cd/m2]')

#x1 = np.linspace(0.000000000000001,1,1000000)

axL.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
axL.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
axL.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
axL.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
axL.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)



axL.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
axL.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
axL.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)


#value_counter_image_luma = split_array_to_stops(axI_Display_Linear_Line_Luma[:,0])
value_counter_image_hlg_FR = split_array_to_stops(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0.005,1000))
value_counter_image_pq_FR = split_array_to_stops(c.models.eotf_ST2084(legal2full(np.linspace(78,722,645))))

#print(value_counter_image_luma)

bars = ('A', 'B', 'C', 'D', 'E',
    'F','G', 'H', 'I', 'J', 
    'K','L','M', 'N', 'O', 
    'P', 'Q','R','A', 'B', 
    'C', 'D', 'E','F','G', 
    'H', 'I', 'J', 'K','L',
    'M', 'N', 'O', 'P', 'Q' , 'R')

y_pos = np.arange(len(bars))

axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)

#axL.bar(y_pos[1:35]/2 + 0.15 + (0.1) + 1, np.divide(value_counter_image_luma[1:35],sum(value_counter_image_luma)), color="black", width=0.3, alpha=0.3, label="Image Display Linear Light Luma")
#axL.bar(y_pos[0:1]/2 + 0.15 + (0.1) + 1 + 0.1, np.divide(value_counter_image_luma[0:1],sum(value_counter_image_luma)), color="black", width=0.1, alpha=0.3)
#axL.bar(y_pos[35:36]/2 + 0.15 + (0.1) + 1 - 0.125, np.divide(value_counter_image_luma[35:36],sum(value_counter_image_luma)), color="black", width=0.1, alpha=0.3)

#axL.bar(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), color="black", width=0.5, alpha=0.3, label="HLG EOTF 10 bit Narrow Range")
#axL.bar(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_pq_FR)), color="black", width=0.5, alpha=0.3, label="PQ EOTF 10 bit Narrow Range")

y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
y_pos_neu = np.linspace(0,17,18)

axL.set_xticks(y_pos_neu+1.5, y_pos_between)

axL.set_xlim(0.5,19.5)
axL.set_yscale("log")
axL.set_ylim(0.0001,1)
axL.set_yticks([0.0001,0.001,0.01,0.1,1],[0.01,0.1,1,10,100])
